# Storing skymaps

## Read raw skymap files

In [1]:
import pickle
from pathlib import Path
import math
import numpy as np


from lsst.sphgeom import Box, ConvexPolygon, UnitVector3d
from math import pi
from lsst.sphgeom import LonLat, UnitVector3d, ConvexPolygon
import yaml
from lsst.sphgeom import UnitVector3d, ConvexPolygon

In [2]:
home_dir = "/sdf/home/o/olynn/"
raw_skymaps_dir = Path(home_dir, "skymap-to-poly-coords", "tests", "data", "raw-skymaps")

skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"
with open(skymap_path, "rb") as f:
    lsst_skymap = pickle.load(f)
lsst_skymap

## Utility functions

In [7]:
def box_to_convex_polygon(box: Box) -> ConvexPolygon:
    if box.isEmpty():
        raise ValueError("Cannot convert an empty Box to a ConvexPolygon.")

    # Get the corners of the box
    lon_a, lon_b = box.getLon().getA().asRadians(), box.getLon().getB().asRadians()
    lon_min = min(lon_a, lon_b)
    lon_max = max(lon_a, lon_b)
    lat_a, lat_b = box.getLat().getA().asRadians(), box.getLat().getB().asRadians()
    lat_min = min(lat_a, lat_b)
    lat_max = max(lat_a, lat_b)
    # todo : this may be an improper assumption, considering RA wrap around!!

    bottom_left = LonLat.fromRadians(lon_min, lat_min)
    bottom_right = LonLat.fromRadians(lon_max, lat_min)
    top_right = LonLat.fromRadians(lon_max, lat_max)
    top_left = LonLat.fromRadians(lon_min, lat_max)

    # Convert corners to UnitVector3d
    vertices = [
        UnitVector3d(bottom_left),
        UnitVector3d(bottom_right),
        UnitVector3d(top_right),
        UnitVector3d(top_left),
    ]

    # Create and return the ConvexPolygon
    return ConvexPolygon(vertices)


def get_poly_from_tract_id(tract_id, inner=False) -> ConvexPolygon:
    tract = lsst_skymap.generateTract(tract_id)
    if inner:
        res = tract.inner_sky_region
    else:
        res = tract.outer_sky_polygon
    if isinstance(res, Box):
        res = box_to_convex_polygon(res)
    return res


def point_in_poly(polygon, ra_degrees, dec_degrees):
    vec = UnitVector3d(LonLat.fromDegrees(ra_degrees, dec_degrees))
    return polygon.contains(vec)

point_in_poly(get_poly_from_tract_id(1), 0.0, -88.0)


def polys_are_equiv(poly_a, poly_b, rtol=1e-12, atol=1e-14):
    """Check if two ConvexPolygons are equivalent within floating point tolerance.

    Parameters
    ----------
    poly_a, poly_b : sphgeom.ConvexPolygon
        The polygons to compare.
    rtol : float
        Relative tolerance for np.allclose.
    atol : float
        Absolute tolerance for np.allclose.

    Returns
    -------
    bool
        True if all vertices match within tolerance.
    """
    verts_a = poly_a.getVertices()
    verts_b = poly_b.getVertices()

    if len(verts_a) != len(verts_b):
        return False

    return np.allclose(verts_a, verts_b, rtol=rtol, atol=atol)

## Read inner_poly and outer_poly

In [3]:
def load_polygons(yaml_path):
    """Load exact inner or outer polygons from a YAML file using 3D unit vectors.

    Parameters
    ----------
    yaml_path : str
        Path to the YAML file written by `write_polygons`.

    Returns
    -------
    dict
        Mapping from tract ID (int) to sphgeom.ConvexPolygon.
    """
    with open(yaml_path, "r") as f:
        data = yaml.safe_load(f)

    poly_dict = {}

    for tract_id_str, vec_list in data["tracts"].items():
        tract_id = int(tract_id_str)

        unit_vecs = [UnitVector3d(*vec) for vec in vec_list]

        # Skip degenerate polygons (fewer than 3 unique vertices)
        unique_vecs = {tuple(round(x, 12) for x in v) for v in unit_vecs}
        if len(unique_vecs) < 3:
            print(f"⚠️ Skipping degenerate tract {tract_id}")
            continue

        poly = ConvexPolygon(unit_vecs)
        poly_dict[tract_id] = poly

    print(f"✅ Loaded {len(poly_dict)} polygons from {yaml_path}")
    return poly_dict

In [13]:
skymap_out_dir = "/sdf/home/o/olynn/skymap-to-poly-coords/skymaps_out/"
inner_poly_path = Path(skymap_out_dir) / "inner_polys.yaml"
outer_poly_path = Path(skymap_out_dir) / "outer_polys.yaml"

inner_poly_map = load_polygons(inner_poly_path)
outer_poly_map = load_polygons(outer_poly_path)

⚠️ Skipping degenerate tract 0
⚠️ Skipping degenerate tract 18937
✅ Loaded 18936 polygons from /sdf/home/o/olynn/skymap-to-poly-coords/skymaps_out/inner_polys.yaml
✅ Loaded 18938 polygons from /sdf/home/o/olynn/skymap-to-poly-coords/skymaps_out/outer_polys.yaml


In [14]:
print(len(inner_poly_map))
print(len(outer_poly_map))


18936
18938


In [8]:
# Just patch the polar caps in for now (todo)

inner_poly_map[0] = get_poly_from_tract_id(0, inner=True)
inner_poly_map[len(inner_poly_map)-1] = get_poly_from_tract_id(len(inner_poly_map)-1, inner=True)

In [12]:
len(inner_poly_map)-1

18936

## Check our saved-and-loaded tracts against the tracts we read via the lsst.skymap package

In [9]:
#tracts_to_check = np.linspace(0, lsst_skymap._numTracts-1, 1000, dtype=int)
tracts_to_check = range(0, lsst_skymap._numTracts)

for tract_id in tracts_to_check:
    ground_truth_poly = get_poly_from_tract_id(tract_id, inner=True)
    loaded_poly = inner_poly_map[tract_id]
    if not polys_are_equiv(ground_truth_poly, loaded_poly):
        print(f"Tract {tract_id} polygons are NOT equivalent!")
    else:
        continue

KeyError: 18937